## 0.0. setup

1. follow the setup instructions in `1_0_hybridization_setup.ipynb`

2. create directories

In [1]:
!mkdir -p ~/hybridization_data/databases_hybrid

## 0.1. imports
### 0.1.1. regular imports

In [2]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import brightway2 as bw
import ecospold2matrix as e2m
import pymrio
# data science
import pandas as pd
# deep copy
import copy

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/weinold/bw_data


### 0.1.2. local imports

In [3]:
sys.path.append(os.path.join(Path.home(), 'pylcaio', 'src')) # required for local import of pylcaio
import pylcaio

## 0.2. file paths
### 0.2.1. directories

In [4]:
%%capture
print(path_dir_databases_pickle := os.path.join(Path.home(), 'hybridization_data/databases_pickle'))
print(path_dir_databases_hybrid := os.path.join(Path.home(), 'hybridization_data/databases_hybrid'))

### 0.2.2. databases

In [5]:
%%capture
# Exiobase
print(path_file_exiobase := os.path.join(path_dir_databases_pickle, 'exiobase_monetary_pxp_2012.pickle'))
# Ecoinvent
print(path_file_ecoinvent := os.path.join(path_dir_databases_pickle, 'ecoinvent_3_5_cutoffPandas_symmNorm.gz.pickle'))
# Hybrid
print(name_file_hybrid := 'hybrid_ecoinvent_3_5_exiobase_monetary_pxp_2012.pickle')
print(path_file_hybrid := os.path.join(path_dir_databases_hybrid, name_file_hybrid))

## 0.3. load `pickle` files

In [6]:
with gzip.open(path_file_ecoinvent, 'rb') as filestream:
    ecoinvent: dict = pd.read_pickle(filestream)

In [7]:
with open(path_file_exiobase, 'rb') as filestream:
    exiobase: pymrio.core.mriosystem.IOSystem = pd.read_pickle(filestream)

## 1.1. `pylcaio` hybridization

### 1.1.1. hybridization

In [8]:
database_loader: pylcaio.DatabaseLoader  = pylcaio.DatabaseLoader(
    lca_database_processed = ecoinvent,
    io_database_processed = exiobase,
    lca_database_name_and_version = 'ecoinvent3.5',
    io_database_name_and_version = 'exiobase3'
)

In [9]:
lcaio_object: pylcaio.LCAIO = database_loader.combine_ecoinvent_exiobase(
    complete_extensions = False,
    impact_world = False,
    regionalized = False
)

No path for the capital folder was provided. Capitals will not be endogenized


In [10]:
lcaio_object = copy.deepcopy(lcaio_object) # to avoid errors when re-running cells
lcaio_object.hybridize(
    price_neutral_cut_off_matrix = 'STAM',
    capitals = False,
    priceless_scaling = True
)

Indentifying Rest of World regions...
Updating electricity prices...
Calculating productions volumes...
Adjusting low production volume processes...
Extending inventory...
Building H matrix...


/home/weinold/pylcaio/src/pylcaio.py:874: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.H = self.H.append([self.H] * (self.number_of_countries_IO + self.number_of_RoW_IO - 1))


Building geography concordance...
Filter H matrix...
Build Cut-off matrix...
Add processes with 'priceless scaling' to Cut-off matrix...


### 1.1.2. save files

In [11]:
lcaio_object.save_system( # this method must be used, otherwise pickle files cannot be read correcly by pylcaio.Analysis
    file_name = name_file_hybrid,
    file_path = path_dir_databases_hybrid,
    format = 'pickle'
)

Database saved to /home/weinold/hybridization_data/databases_hybrid/hybrid_ecoinvent_3_5_exiobase_monetary_pxp_2012.pickle
Description file saved to /home/weinold/hybridization_data/databases_hybrid/description_hybrid_ecoinvent_3_5_exiobase_monetary_pxp_2012.txt


## X.1. investigation

In [12]:
lcaio_object.K_io

""


In [13]:
lcaio_object.IMP

,method,category,indicator,impact_score_unit
impact_label,,,,
"EDIP; environmental impact; global warming, GWP 100a; kg CO2-Eq",EDIP,environmental impact,"global warming, GWP 100a",kg CO2-Eq
"EDIP; environmental impact; global warming, GWP 20a; kg CO2-Eq",EDIP,environmental impact,"global warming, GWP 20a",kg CO2-Eq
"EDIP; environmental impact; global warming, GWP 500a; kg CO2-Eq",EDIP,environmental impact,"global warming, GWP 500a",kg CO2-Eq
"EDIP; environmental impact; photochemical ozone formation, high NOx POCP; kg ethyle.",EDIP,environmental impact,"photochemical ozone formation, high NOx POCP",kg ethyle.
"EDIP; environmental impact; photochemical ozone formation, low NOx POCP; kg ethyle.",EDIP,environmental impact,"photochemical ozone formation, low NOx POCP",kg ethyle.
...,...,...,...,...
selected LCI results; water; BOD; kg,selected LCI results,water,BOD,kg
EPS 2000; total; emissions into water; ELU,EPS 2000,total,emissions into water,ELU
"selected LCI results, additional; water, radioactive; tritium; kBq","selected LCI results, additional","water, radioactive",tritium,kBq


In [14]:
ecoinvent['STR']

,id,name,unit,cas,comp,subcomp
id,,,,,,
d21da01e-f96f-4db5-9746-7b70db8a1f2c,d21da01e-f96f-4db5-9746-7b70db8a1f2c,"1,4-Butanediol",kg,000110-63-4,air,"low population density, long-term"
90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,"1,4-Butanediol",kg,000110-63-4,air,lower stratosphere + upper troposphere
83bafcf1-2f2e-4a32-89a0-f1f16ca10626,83bafcf1-2f2e-4a32-89a0-f1f16ca10626,"1,4-Butanediol",kg,000110-63-4,air,non-urban air or from high stacks
09db39be-d9a6-4fc3-8d25-1f80b23e9131,09db39be-d9a6-4fc3-8d25-1f80b23e9131,"1,4-Butanediol",kg,000110-63-4,air,unspecified
38a622c6-f086-4763-a952-7c6b3b1c42ba,38a622c6-f086-4763-a952-7c6b3b1c42ba,"1,4-Butanediol",kg,000110-63-4,air,urban air close to ground
...,...,...,...,...,...,...
e9eb024b-6fba-4dce-b280-9415b2ba4695,e9eb024b-6fba-4dce-b280-9415b2ba4695,t-Butylamine,kg,000075-64-9,water,ground-
d0aaa60f-40d9-4c12-a486-954333aa549f,d0aaa60f-40d9-4c12-a486-954333aa549f,t-Butylamine,kg,000075-64-9,water,"ground-, long-term"
cc415ec5-ef9b-4239-b423-32906818e457,cc415ec5-ef9b-4239-b423-32906818e457,t-Butylamine,kg,000075-64-9,water,ocean


In [15]:
ecoinvent.keys()

dict_keys(['STR', 'IMP', 'PRO_header', 'STR_header', 'IMP_header', 'processingdata'])